In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/MyDrive/Colab Notebooks/data_ats_1/images'
test_data_path = '/content/drive/MyDrive/Colab Notebooks/data_ats_1/test_images'
folders = ['artificial roughness', 'give way', 'movement prohibition', 'no entry', 'parking', 
           'pedestrian crossing', 'road works', 'stop']

## Предобработка данных

In [5]:
import os
import glob

files = {}
for f in folders:
  files[f] = glob.glob(os.path.join(data_path, f, '*.jpg'))

In [6]:
import cv2
img = cv2.imread(files[f][0]) 
img[0][0]

array([  0,  88, 139], dtype=uint8)

In [7]:
from PIL import Image, ImageEnhance, ImageOps
from matplotlib import cm

def preprocess_image(img):
  img = cv2.resize(img, (50, 50))
  pil_img = Image.fromarray((img * 255).astype(np.uint8))
  enhancer = ImageEnhance.Contrast(pil_img)
  factor = 2
  pil_img = enhancer.enhance(factor)
  pil_img = ImageOps.invert(pil_img)
  img = np.asarray(pil_img)
  img = cv2.GaussianBlur(img, (3, 3), 5)
  return img
  # enhance contrast

In [8]:
import cv2
from google.colab.patches import cv2_imshow 

data = pd.DataFrame(columns=['image', 'label'])
images, labels = [], []
for f in folders:
  for cnt, i in enumerate(files[f]):
    img = cv2.imread(i)
    img = preprocess_image(img)
    #cv2_imshow(img)
    images.append(np.asarray(img).astype('float32'))
    labels.append(f)
  print(f, 'folder processed')
data['image'] = pd.Series(images)
data['label'] = pd.Series(labels)

artificial roughness folder processed
give way folder processed
movement prohibition folder processed
no entry folder processed
parking folder processed
pedestrian crossing folder processed
road works folder processed
stop folder processed


In [9]:
new_imgs = images.copy()
new_imgs = np.asarray(new_imgs)
new_imgs.shape

In [10]:
def one_hot_encode(label):
    one_hot_label_dictionary = {"no entry": [1, 0, 0, 0, 0, 0, 0, 0],
                                "pedestrian crossing": [0, 1, 0, 0, 0, 0, 0, 0],
                                "road works": [0, 0, 1, 0, 0, 0, 0, 0],
                                "movement prohibition": [0, 0, 0, 1, 0, 0, 0, 0],
                                "parking": [0, 0, 0, 0, 1, 0, 0, 0],
                                "stop": [0, 0, 0, 0, 0, 1, 0, 0],
                                "give way": [0, 0, 0, 0, 0, 0, 1, 0],
                                "artificial roughness": [0, 0, 0, 0, 0, 0, 0, 1]}

    one_hot_encoded = one_hot_label_dictionary[label]
    return one_hot_encoded

In [12]:
new_labels = np.asarray(labels).reshape((-1, 1))
new_labels.shape

(2481, 1)

In [13]:
new_labels

array([['artificial roughness'],
       ['artificial roughness'],
       ['artificial roughness'],
       ...,
       ['stop'],
       ['stop'],
       ['stop']], dtype='<U20')

In [16]:
new_l = np.empty((2481, 8))
for i in range(len(new_labels)):
  new_l[i] = one_hot_encode(new_labels[i][0])

In [17]:
new_l

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(new_imgs, new_l, test_size=0.2, random_state=56)
y_train.shape

(1984, 8)

In [19]:
X_train.shape

(1984, 50, 50, 3)

## Модель LeNet-5 на Keras

In [20]:
class LeNet:
  def build(df_size, channels, rows, cols, classes, activation='relu'):
    if K.image_data_format() == "channels_first":
      input_shape = (channels, rows, cols, df_size)
    else:
      input_shape = (rows, cols, channels)
    
    model = Sequential([
            Conv2D(20, 5, padding="same", input_shape=input_shape),
            Activation(activation),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Conv2D(50, 5, padding="same"),
            Activation(activation),
            MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
            Flatten(),
            Dense(500),
            Activation(activation),
            Dense(classes),
            Activation('softmax')
    ])

    return model

In [21]:
def recall_metrics(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_metrics(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_metrics(y_true, y_pred):
    precision = precision_metrics(y_true, y_pred)
    recall = recall_metrics(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [22]:
model = LeNet.build(2481, 3, 50, 50, 8)

In [23]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', f1_metrics])

In [24]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [25]:
history = model.fit(X_train, y_train, epochs=10, batch_size=20, validation_data=(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4527: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/10
100/100 [==============================] - 22s 211ms/step - loss: 0.9179 - accuracy: 0.8165 - f1_metrics: 0.8165 - val_loss: 0.0879 - val_accuracy: 0.9256 - val_f1_metrics: 0.9278
Epoch 2/10
100/100 [==============================] - 20s 205ms/step - loss: 0.0217 - accuracy: 0.9814 - f1_metrics: 0.9817 - val_loss: 0.0128 - val_accuracy: 0.9920 - val_f1_metrics: 0.9919
Epoch 3/10
100/100 [==============================] - 21s 213ms/step - loss: 0.0060 - accuracy: 0.9970 - f1_metrics: 0.9972 - val_loss: 0.0229 - val_accuracy: 0.9759 - val_f1_metrics: 0.9770
Epoch 4/10
100/100 [==============================] - 21s 207ms/step - loss: 0.0030 - accuracy: 0.9990 - f1_metrics: 0.9990 - val_loss: 0.0134 - val_accuracy: 0.9899 - val_f1_metrics: 0.9910
Epoch 5/10
100/100 [==============================] - 21s 210ms/step - loss: 0.0041 - accuracy: 0.9980 - f1_metrics: 0.9980 - val_loss: 0.0164 - val_accuracy: 0.9859 - val_f1_metrics: 0.9879
Epoch 6/10
100/100 [=========================

In [26]:
def standardize_output(label):
    out = [0 for _ in range(8)]
    label = list(label)
    id = label.index(max(label)) 
    out[id] = 1
    print(out)
    return out

In [ ]:
y_pred = model.predict(X_val)
y_st = []
for label in y_pred:
  y_st.append(standardize_output(label))

In [28]:
true_cnt = 0
for true_label, pred_label in zip(y_val, y_st):
  if list(true_label) == pred_label:
    true_cnt += 1
print('Точность:', true_cnt / len(y_val))

Точность: 0.9698189134808853


## Сохранение нейронки

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/data_ats_2/model_keras_1')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/data_ats_2/model_keras_1/assets


## Загрузка модели Keras

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/data_ats_2/model_keras_1')